# EDA Marine Microplastics 🌊

## Importación de librerías y datos 📁

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [ ]:
df_mp = pd.read_csv("../files/transition_files/Marine_microplastics/Marine_Microplastics.csv")
df_mp.head()

,OBJECTID,Oceans,Regions,SubRegions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,x,y
0,9676,Atlantic Ocean,NaN,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696000,-48.560000,8/11/2015 12:00:00 AM,a77121b2-e113-444e-82d9-7af11d62fdd2,-48.560000,-31.696000
1,6427,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350000,-121.850000,12/18/2002 12:00:00 AM,be27c450-02ca-4261-8d89-cae21108e6cc,-121.850000,6.350000
2,10672,Pacific Ocean,NaN,NaN,Manta net,0.013,pieces/m3,0.005-1,Medium,Goldstein et al.2013,"Goldstein, M.C., A.J. Titmus, M. Ford. 2013. S...",https://doi.org/10.1371/journal.pone.0080020,Scripps Institution of Oceanography-University...,Great Pacific Garbage Patch/SEAPLEX,253448,https://www.ncei.noaa.gov/access/metadata/land...,0.500000,-95.350000,10/17/2006 12:00:00 AM,23effcdd-35b7-4e1e-adb4-390693a287d3,-95.350000,0.500000
3,13921,Atlantic Ocean,NaN,NaN,Aluminum bucket,1368.000,pieces/m3,>=10,Very High,Queiroz et al.2022,"Queiroz, A.F.dos S., A.S. da Conceição, D. Che...",https://doi.org/10.1016/j.scitotenv.2022.156259,"Federal University of Pará, Brazil",Amazon Continental Shelf,276482,https://www.ncei.noaa.gov/access/metadata/land...,0.631825,-45.398158,10/17/2018 12:00:00 AM,16d77822-0533-4116-97b9-0bdb592f3d6e,-45.398158,0.631825
4,9344,Pacific Ocean,NaN,NaN,Grab sample,0.001,pieces/m3,0.0005-0.005,Low,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,16.623000,-99.697800,1/3/2015 12:00:00 AM,b9e435e3-9e86-4143-8b51-877e5dcdc7a6,-99.697800,16.623000


## Primera exploración 🔎

In [ ]:
# pip install geopandas

In [4]:
df_mp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20425 entries, 0 to 20424
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OBJECTID          20425 non-null  int64  
 1   Oceans            20154 non-null  object 
 2   Regions           8885 non-null   object 
 3   SubRegions        1307 non-null   object 
 4   Sampling Method   20425 non-null  object 
 5   Measurement       14613 non-null  float64
 6   Unit              20425 non-null  object 
 7   Density Range     20425 non-null  object 
 8   Density Class     20425 non-null  object 
 9   Short Reference   20425 non-null  object 
 10  Long Reference    20425 non-null  object 
 11  DOI               20425 non-null  object 
 12  Organization      20425 non-null  object 
 13  Keywords          20407 non-null  object 
 14  Accession Number  20425 non-null  int64  
 15  Accession Link    20425 non-null  object 
 16  Latitude          20425 non-null  float6

## Gestión de duplicados y nulos ✏️

In [6]:
# Función para conocer nulos y duplicados en un informe. Next step--> ETL

def nulos_duplicados(df_mp):
    # Cálculo del porcentaje de nulos
    porcentaje_nulos = df_mp.isna().sum() / df_mp.shape[0] * 100
    
    # Verificación de duplicados
    duplicados = df_mp.duplicated().sum()
    if duplicados == 0:
        mensaje_duplicados = "No hay duplicados"
    else:
        mensaje_duplicados = f"Hay {duplicados} duplicados"
    
    # Informe visual
    informe = f"""
    ===================== Informe de Datos =====================
    
    Porcentaje de Nulos por Columna:
    ------------------------------------------------------------
    {porcentaje_nulos.to_string()}
    
    ------------------------------------------------------------
    Duplicados:
    ------------------------------------------------------------
    {mensaje_duplicados}
    
    ============================================================
    """
    
    # Imprimir directamente el reporte
    print(informe)

# Ejemplo de uso
# df_mp = pd.DataFrame(...)

# Llamar directamente a la función
nulos_duplicados(df_mp)


    ===================== Informe de Datos =====================
    
    Porcentaje de Nulos por Columna:
    ------------------------------------------------------------
    OBJECTID             0.000000
Oceans               1.326805
Regions             56.499388
SubRegions          93.600979
Sampling Method      0.000000
Measurement         28.455324
Unit                 0.000000
Density Range        0.000000
Density Class        0.000000
Short Reference      0.000000
Long Reference       0.000000
DOI                  0.000000
Organization         0.000000
Keywords             0.088127
Accession Number     0.000000
Accession Link       0.000000
Latitude             0.000000
Longitude            0.000000
Date                 0.000000
GlobalID             0.000000
x                    0.000000
y                    0.000000
    
    ------------------------------------------------------------
    Duplicados:
    ------------------------------------------------------------
    No hay d

In [ ]:
# Gestión de duplicados: No procede. No hay duplicados

In [ ]:
# Gestión de nulos:

#Comprobación de nulos para la columna 'Oceans' de un dataframe:
print(f"Número de puntos con 'Oceans' nulo: {df_mp['Oceans'].isna().sum()}")

In [ ]:
#Conocer las columnas de la capa .shp que usaremos en shapely

# OJO! NO ACCESIBLE SI NO ESTÁN LOS ARCHIVOS DESCARGADOS
# MÁS INFO EN EL repositorio: Capas_geográficas.txt
print(ocean_shapes.columns)

In [7]:
import geopandas as gpd
from shapely.geometry import Point

# Primero, seleccionamos los puntos que son nulos, en nuestra columna "Oceans"
df_nulos = df_mp[df_mp['Oceans'].isna()].copy()

# Creamos una geometría, con ayuda de nuestras columnas Lon y Lat (donde no hay nulos)
df_nulos['geometry'] = df_nulos.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

# Creamos el "GeoDataFrame" para puntos nulos
gdf_nulos = gpd.GeoDataFrame(df_nulos, geometry='geometry', crs="EPSG:4326")

# Cargamos un archivo shapefile, una capa con información sobre los oceanos
# Este shapefle tiene esta info:
#'name', 'latitude', 'longitude', 'min_Y', 'min_X', 'max_Y', 'max_X','area_km2', 'geometry'
ocean_shapes = gpd.read_file("../archivos/goas_v01.shp")

# Realizamos la unión espacial más cercana con sjoin.nearest:
gdf_nulos_nearest = gpd.sjoin_nearest(gdf_nulos, ocean_shapes, how='left', distance_col='dist')

# Asignamos el nombre correcto de los océanos a la columna 'Oceans' que tiene relación con la unión.
gdf_nulos_nearest['Oceans'] = gdf_nulos_nearest['name']  

# Ahora actualizamos el DataFrame original 'df' con los valores de 'Oceans'
df_mp.loc[gdf_nulos_nearest.index, 'Oceans'] = gdf_nulos_nearest['Oceans']

# Volvemos a comprobar los puntos con oceano nulo, en 'Oceans:
print(f"Número de puntos con 'Oceans' nulo: {df_mp['Oceans'].isna().sum()}")


c:\Users\isaw9\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


Número de puntos con 'Oceans' nulo: 0


In [ ]:
#Conocer las columnas de mi capa .shp
print(ocean_shapes.columns)

In [ ]:
#comprobación de que la herramienta de geopandas, ha funcionado. Este océano ha sido "generado"
df_mp[df_mp['OBJECTID'] == 19864]

In [ ]:
# Comprobamos los valores únicos para 'Oceans'
df_mp['Oceans'].unique()

In [ ]:
# Cómo tienen mayor detalle geográfico del esperado, convertimos el resultado para obtener una región más amplia y acorde a los datos
df_mp=df_mp.replace("North Atlantic Ocean", "Atlantic Ocean")
df_mp=df_mp.replace("North Pacific Ocean","Pacific Ocean")

In [ ]:
# Volvemos a comprobar los valores únicos para 'Oceans'
df_mp['Oceans'].unique()

In [ ]:
#HACEMOS LO MISMO PARA REGIONES:

# Seleccionar solo los puntos nulos en 'Oceans'
df_nulos = df_mp[df_mp['Regions'].isna()].copy()

# Crear geometría a partir de latitud y longitud
df_nulos['geometry'] = df_nulos.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

# Crear el GeoDataFrame para puntos nulos
gdf_nulos = gpd.GeoDataFrame(df_nulos, geometry='geometry', crs="EPSG:4326")

# Cargar el shapefile con las formas de los océanos
ocean_shapes = gpd.read_file("../../Archivos/goas_v01.shp")

# Realizar la unión espacial más cercana (nearest)
gdf_nulos_nearest = gpd.sjoin_nearest(gdf_nulos, ocean_shapes, how='left', distance_col='dist')

# Asignar el nombre correcto de los océanos a la columna 'Oceans'
gdf_nulos_nearest['Regions'] = gdf_nulos_nearest['name']  # o el nombre correcto de la columna en ocean_shapes

# Ahora actualizamos el DataFrame original 'df' con los valores de 'Oceans'
df_mp.loc[gdf_nulos_nearest.index, 'Regions'] = gdf_nulos_nearest['Regions']

# Imprimir el número de valores nulos en la columna 'Oceans'
print(f"Número de puntos con océano nulo: {df_mp['Regions'].isna().sum()}")

In [ ]:
# Comprobaciones varias:
df_mp["Unit"].unique()

array(['pieces/m3', 'pieces kg-1 d.w.', 'pieces/10 mins'], dtype=object)

In [ ]:
# Comprobaciones varias:
# saber a que método corresponden las medidas
# Tabla de contingencia entre Métodos y Unidades de Medida
contingencia = pd.crosstab(df_mp['Sampling Method'], df_mp['Unit'], margins=True)
display(contingencia)

Unit,pieces kg-1 d.w.,pieces/10 mins,pieces/m3,All
Sampling Method,,,,
AVANI net,0,0,18,18
Aluminum bucket,0,0,57,57
CTD rosette sampler,0,0,36,36
Day grab,17,0,0,17
Grab sample,0,0,1181,1181
Hand picking,0,5812,18,5830
Intake seawater pump,0,0,181,181
Manta net,0,0,2342,2342
Megacorer,90,0,0,90


In [ ]:
# Se puede observar, que excepto "hand picking" las demás tienen una unidad definida.

#Como tenemos nulos en measurement, no en unit.

In [ ]:
# Comprobaciones varias:

df_mp.groupby("Sampling Method")["Measurement"].apply(lambda x: x.isnull().sum())
# Podemos ver que todos los nulos son de Hand picking, "recogido a mano"--> No relevante --> No imputación

Sampling Method
AVANI net                       0
Aluminum bucket                 0
CTD rosette sampler             0
Day grab                        0
Grab sample                     0
Hand picking                 5812
Intake seawater pump            0
Manta net                       0
Megacorer                       0
Metal spoon                     0
Neuston net                     0
PVC cylinder                    0
Petite Ponar benthic grab       0
Plankton net                    0
Remotely operated vehicle       0
Shipek grab sampler             0
Stainless steel spoon           0
Van Dorn sampler                0
Van Veen grab sampler           0
Name: Measurement, dtype: int64

## Transformaciones 💻

In [ ]:
# Transformaciones:

In [ ]:
# En la columna Density range, tengo un rango. Quedarme con el valor central, para próximos cálculos. Pero sin eliminar la columna original

""" FINALMENTE, USAMOS EL VALOR DE Measurement. Pero podria ser útil la creación de esta columna
para promediar datos, igualmente dejamos el proceso"""

In [ ]:
df_mp["Density Range"].unique()

array(['0.005-1', '0-0.0005', '>=10', '0.0005-0.005', '1-10', '0-2', '0',
       '2-40', '40-200', '500-30000', '0-100', '1-2', '2-20', '>200',
       '20-150', '>40000', '150-200', '30000-40000'], dtype=object)

In [ ]:
# Función para eliminar el símbolo '>= y >'
def eliminar_menor_igual(rango):
    return rango.replace('>=','').replace('>','').strip()

# Aplicamos la función para eliminar '>=' de la columna 'Density Range'
df_mp['Density Range'] = df_mp['Density Range'].apply(eliminar_menor_igual)

In [ ]:
df_mp["Density Range"].unique()

array(['0.005-1', '0-0.0005', '10', '0.0005-0.005', '1-10', '0-2', '0',
       '2-40', '40-200', '500-30000', '0-100', '1-2', '2-20', '200',
       '20-150', '40000', '150-200', '30000-40000'], dtype=object)

In [ ]:
# Función para extraer los valores numéricos y calcular el valor central
def calcular_densidad_central(rango):
    # Si el valor es solo un número
    if '-' not in rango:  # Caso cuando no hay guion, es un solo número
        return float(rango.strip())
    
    # Si el valor es un rango (con '-')
    else:
        # Extraemos los valores del rango y calculamos el promedio
        min_val, max_val = map(float, rango.replace(' ', '').split('-'))  # Convertimos los valores en float
        return (min_val + max_val) / 2  # Calculamos el promedio del rango

# Aplicamos la función a la columna 'Density Range' y creamos la nueva columna 'Density_Center'
df_mp['Density_Center'] = df_mp['Density Range'].apply(calcular_densidad_central)


In [ ]:
# Agrupar por 'Oceans' y contar las ocurrencias de cada 'Regions'
oceans_regions_count = df_mp.groupby('Oceans')['Regions'].value_counts()

# Encontrar la región más frecuente en cada océano
most_frequent_region = oceans_regions_count.groupby('Oceans').idxmax()

# Contar la frecuencia de la región más frecuente
most_frequent_count = oceans_regions_count.groupby('Oceans').max()

# Calcular el total de registros por océano
total_by_ocean = df_mp.groupby('Oceans').size()

# Calcular el porcentaje de la región más frecuente dentro de cada océano
percentage_most_frequent = (most_frequent_count / total_by_ocean) * 100

# Crear un DataFrame con la región más frecuente y su porcentaje
result = pd.DataFrame({
    'Most Frequent Region': most_frequent_region,
    'Frequency': most_frequent_count,
    'Percentage': percentage_most_frequent
})

# Mostrar el resultado
print(result)


                               Most Frequent Region  Frequency  Percentage
Oceans                                                                    
Arctic Ocean          (Arctic Ocean, greenland sea)         35   19.662921
Atlantic Ocean     (Atlantic Ocean, gulf of mexico)       4817   31.083436
Indian Ocean     (Indian Ocean, mozambique channel)          3   15.000000
Pacific Ocean   (Pacific Ocean, gulf of california)        116    2.452431


In [ ]:
# Guardar csv filtrado

# df_mp.to_csv('../files/transition_files/Marine_microplastics/01_Marine_Microplastics_all_columns_EDA.csv', index=False)